<a href="https://colab.research.google.com/github/Suchetaaa/Boolean-Arithmetic-SAGE/blob/master/Caching_Bandits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setting Definitions and trial experiments
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

num_arms = 10

success_probs = np.sort([np.random.rand() for i in range(num_arms)])[::-1]
# print(success_probs)
costs = np.sort([np.random.rand() for i in range(num_arms)])
success_cumsum = np.cumsum(success_probs)

metrics = [(success_cumsum[i]-costs[i])/(i+1) for i in range(num_arms)]
print(metrics)

print(np.argmax(metrics),np.max(metrics))
print("************Trial Experiment**************")

arm_to_pull = 2
t = 0
T = 100000
reward = []
while t<T:
    for i in range(arm_to_pull):
      r = 1 if np.random.rand()<=success_probs[i] else 0
      reward.append(r)
    reward[-1] -= costs[arm_to_pull-1]
    t+=arm_to_pull
print(np.cumsum(reward)[-1]/100000,metrics[arm_to_pull-1])

In [ ]:


class Arm:
  def __init__(self):
    self.pulls = 0
    self.rewards = 0
  def update(self,reward):
    self.pulls+=1
    self.rewards+=reward
  def estimate(self):
    assert self.pulls>0
    return self.rewards/self.pulls

class Oracle:
  def __init__(self,M,p,c):
    self.M = M
    self.prob_estimates = [0 for i in range(self.M)]
    self.c = c
    self.g = np.cumsum(p)
    self.optimal_arm = np.argmax([(self.g[i]-self.c[i])/(i+1) for i in range(M)])
    print("Optimal Arm is ",self.optimal_arm)
    # print("Average Utility Calculated at ",np.max([(self.g[i]-self.c[i])/(i+1) for i in range(M)]))

  def update(self,reward,arm):
    return
  
  def reset(self):
    return
  
  def schedule(self,time):
    return self.optimal_arm

class UCB:
  def __init__(self,M,c):
    self.M = M
    self.prob_estimates = [0 for i in range(self.M)]
    self.g_estimates = np.cumsum(self.prob_estimates)
    self.c = c
    self.arms = [Arm() for i in range(self.M)]
  
  def update(self,reward,arm):
    self.arms[arm].update(reward)
    # if (arm==0): 
      # print("reward",reward)
      # print(self.arms[arm].estimate())
    return
  
  def reset(self):
    self.prob_estimates = [0 for i in range(self.M)]
    self.arms = [Arm() for i in range(self.M)]
    return

  def schedule(self,time):
    if time<=self.M: 
      return self.M-1
    else:
      for arm_i in range(self.M):
        self.prob_estimates[arm_i] = self.arms[arm_i].estimate()
      # print(self.prob_estimates)
      self.g_estimates = np.cumsum(self.prob_estimates)
      self.metric_estimates = [((self.g_estimates[i]-self.c[i])/(i+1)) + np.sqrt(2*np.log(time)/(self.arms[i].pulls*(i+1))) for i in range(self.M)]
      # print(self.metric_estimates)
      # if(np.argmax(metrics)!=np.argmax(metrics)): print(np.argmax(self.metric_estimates))
      # print(np.argmax(self.metric_estimates))
      # if(np.argmax(self.metric_estimates) == 0): print(np.argmax(self.metric_estimates))
      return np.argmax(self.metric_estimates)

class CUCB:
  def __init__(self,M,c):
    self.M = M
    self.prob_estimates = [0 for i in range(self.M)]
    self.g_estimates = np.cumsum(self.prob_estimates)
    self.pseudo_estimates = [[0 for i in range(self.M)] for i in range(self.M)]
    self.c = c
    self.arms = [Arm() for i in range(self.M)]
  
  def update(self,reward,arm):
    self.arms[arm].update(reward)
    return
  
  def reset(self):
    self.prob_estimates = [0 for i in range(self.M)]
    self.arms = [Arm() for i in range(self.M)]
    self.g_estimates = np.cumsum(self.prob_estimates)
    self.pseudo_estimates = [[0 for i in range(self.M)] for i in range(self.M)]
    return
  
  def psuedo_update(self,realizations,m):
    g_m = sum(realizations)
    for n in range(arm):
      self.pseudo_estimates[n][m] = ((self.pseudo_estimates[n][m])*(self.arms[m].pulls-1) + (g_m - (m-n)*self.prob_estimates[n] + c[m] - c[n]))/self.arms[m].pulls
    for n in range(arm+1, self.M):
      self.pseudo_estimates[n][m] = ((self.pseudo_estimates[n][m])*(self.arms[m].pulls-1) + (g_m + (n-m)*realizations[-1:] + c[m] - c[n]))/self.arms[m].pulls
    return

  def schedule(self,time):
    if time<=self.M: 
      return self.M-1
    else:
      arm_pulls = [self.arms[i].pulls for i in range(self.M)]
      most_pulled = np.argmax(arm_pulls)

      competitive_set = set()
      for i in range(self.M): competitive_set.add(i)

      for arm_i in range(self.M):
        self.prob_estimates[arm_i] = self.arms[arm_i].estimate()

      for i in range(self.M):
        if (i != most_pulled and self.prob_estimates[most_pulled] > self.pseudo_estimates[i][most_pulled]):
          competitive_set.remove(i)
      
      self.g_estimates = np.cumsum(self.prob_estimates)
      self.metric_estimates = [((self.g_estimates[i]-self.c[i])/(i+1)) + np.sqrt(2*np.log(time)/(self.arms[i].pulls*(i+1))) for i in range(self.M)]
      max_metric = 0
      max_arm = 0
      for i in competitive_set:
        if (self.metric_estimates[i] > max_metric):
          max_metric = self.metric_estimates[i]
          max_arm = i

      return max_arm


class ETC:
  def __init__(self,M,c,K):
    self.M = M
    self.prob_estimates = [0 for i in range(self.M)]
    self.c = c
    self.arms = [Arm() for i in range(self.M)]
    self.explore_time = K #Will explore for K*M time slots
    self.calced = False
    self.best = -1

  def update(self,reward,arm):
    self.arms[arm].update(reward)
    return
  
  def reset(self):
    self.prob_estimates = [0 for i in range(self.M)]
    self.arms = [Arm() for i in range(self.M)]
    self.calced = False
    self.best = -1
    return
  
  def schedule(self,time):
    if time<self.M*self.explore_time: return self.M-1
    else:
      if self.calced:
        return self.best
      else:
        for i in range(self.M):
          self.prob_estimates[i] = self.arms[i].estimate()
        self.g_estimates = np.cumsum(self.prob_estimates)
        self.metric_estimates = [(self.g_estimates[i]-self.c[i])/(i+1) for i in range(self.M)]
        self.best = np.argmax(self.metric_estimates)
        self.calced = True
        return self.best



In [ ]:
#@title Runner Code
#Probs and Costs defined in the first cell. This cell contains code to run a policy once for T slots and another to do this N times

def run_once(T,M,p,c,policy):
  time = 1
  Rewards = []
  Costs = []
  
  while time<=T:
    played_arm = policy.schedule(time) #Has to return a number in [0,M-1]
    # print(played_arm)
    for slot in range(played_arm+1):
      
      r_slot = 1 if np.random.rand()<=p[slot] else 0
      Rewards.append(r_slot)
      Costs.append(0)
      policy.update(r_slot,slot) #Arm Update Step
    if (policy=="CUCB"):
      policy.pseudo_update(Rewards[-(played_arm+1):],played_arm)
    Costs[-1] = c[played_arm]
    time+=(played_arm+1)
    
  Rewards = np.cumsum(Rewards)[:T]
  Costs = np.cumsum(Costs)[:T]

  return (Rewards-Costs)

def run_n_times(N,T,p,c,policy):
  Utility = np.zeros(T)
  for run in tqdm(range(N)):
    policy.reset()
    M = p.size
    assert p.size==c.size
    Utility += run_once(T,M,p,c,policy)
  return Utility/N


In [ ]:
#@title Simulation Varaiables and Runs
N=1000
T=10000

#Oracle Policy Runs
Oracle_P = Oracle(num_arms,success_probs,costs)
Oracle_Utility = run_n_times(N,T,success_probs,costs,Oracle_P)
plt.plot(Oracle_Utility)
print(Oracle_Utility[-1])

UCB_P = UCB(num_arms,costs)
UCB_Utility = run_n_times(N,T,success_probs,costs,UCB_P)
plt.plot(UCB_Utility)

ETC_P = ETC(num_arms,costs,8*np.log(T))
ETC_Utility = run_n_times(N,T,success_probs,costs,ETC_P)
plt.plot(ETC_Utility)

CUCB_P = CUCB(num_arms,costs)
CUCB_Utility = run_n_times(N,T,success_probs,costs,CUCB_P)
plt.plot(CUCB_Utility)
plt.legend(["Oracle","UCB","ETC","CUCB"])
plt.title("Utility vs Time")


plt.figure()
plt.plot(-UCB_Utility+Oracle_Utility)
plt.plot(-ETC_Utility+Oracle_Utility)
plt.plot(-CUCB_Utility+Oracle_Utility)

plt.legend(["UCB","ETC","CUCB"])
plt.title("Regret vs Time")

Optimal Arm is  0



9110.459900538617


In [ ]:
plt.plot(Oracle_Utility)
plt.plot(UCB_Utility)

ETC_P = ETC(num_arms,costs,10*np.log(T))
ETC_Utility = run_n_times(N,T,success_probs,costs,ETC_P)
plt.plot(ETC_Utility)
plt.legend(["Oracle","UCB","ETC"])
plt.title("Utility vs Time")

plt.figure()
plt.plot(-UCB_Utility+Oracle_Utility)
plt.plot(-ETC_Utility+Oracle_Utility)
plt.legend(["UCB","ETC"])
plt.title("Regret vs Time")

In [ ]:
print(Oracle_Utility)
plt.plot(Oracle_Utility)
plt.plot(UCB_Utility)

ETC_P = ETC(num_arms,costs,5*np.log(T))
ETC_Utility = run_n_times(N,T,success_probs,costs,ETC_P)
plt.plot(ETC_Utility)
plt.legend(["Oracle","UCB","ETC"])
plt.title("Utility vs Time")

plt.figure()
plt.plot(-UCB_Utility+Oracle_Utility)
plt.plot(-ETC_Utility+Oracle_Utility)
plt.legend(["UCB","ETC"])
plt.title("Regret vs Time")
